In [ ]:
!pip3 install --upgrade google-cloud-videointelligence

     |████████████████████████████████| 202 kB 5.2 MB/s 
     |████████████████████████████████| 46 kB 2.3 MB/s 
     |████████████████████████████████| 1.0 MB 32.6 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


Make a copy of 'Systems '22' at your own **MyDrive** 

In [ ]:
%cd gdrive/MyDrive/"Systems '22"/

/content/gdrive/.shortcut-targets-by-id/1ohwBGdC8wri16R_SjnGnOM_B619TDepK/Systems '22


In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
BILLING_ACCOUNT_ID = '01074B-EAE851-71F16B7'

GCP_PROJECT_ID = 'stoked-dominion-341317'
PROJECT_NUMBER = '725342657078'

!gcloud beta billing projects link $GCP_PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.projects.link) PERMISSION_DENIED: Permission denied to enable service [cloudbilling.googleapis.com]
Help Token: Ae-hA1MQIU1vJ1O7evF1rGsPZLmJDh1ST96QJydHIUbyOj3zfNwcTzmODNEpI69gWZFilZ2LSTwNjWDKoHRSbdMD1vw9zDjRCaAPIdWsOv9VI1k_
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:
!gcloud beta billing accounts list

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.accounts.list) PERMISSION_DENIED: Permission denied to enable service [cloudbilling.googleapis.com]
Help Token: Ae-hA1OfPnqieFtMLoXt4UDvIhWbq-HRdtpjx4fQ2YyRPcExoLmiWS8ouGViQANjuIwxc1u18o7MwkTTGiiRQQu6ewl6j2sKN3nZOwHmNZbm18oq
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com

Operation "operations/acat.p2-725342657078-a92e9286-3860-47ca-90cf-dc04cb2fe837" finished successfully.


In [ ]:
SERVICE_ACCOUNT_NAME ='ytnotebook'

SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
#!gcloud iam --project $GCP_PROJECT_ID service-accounts create $SERVICE_ACCOUNT_NAME
!gcloud projects add-iam-policy-binding $GCP_PROJECT_ID \
    --member serviceAccount:$SERVICE_ACCOUNT_EMAIL \
    --role=roles/editor

Updated IAM policy for project [stoked-dominion-341317].
bindings:
- members:
  - user:ylingqin08@gmail.com
  role: roles/billing.projectManager
- members:
  - serviceAccount:725342657078@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-725342657078@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-725342657078@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - serviceAccount:service-725342657078@cloudcomposer-accounts.iam.gserviceaccount.com
  role: roles/composer.serviceAgent
- members:
  - serviceAccount:service-725342657078@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-725342657078@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-725342657078@containerregistry.iam.gserviceac

In [ ]:
DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT = f'service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com'

!gcloud iam --project $GCP_PROJECT_ID service-accounts add-iam-policy-binding \
--role=roles/iam.serviceAccountAdmin \
--member=serviceAccount:$DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT \
$SERVICE_ACCOUNT_EMAIL

API [iam.googleapis.com] not enabled on project [522309567947]. Would you like 
to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [iam.googleapis.com] on project [522309567947]...
ERROR: (gcloud.iam.service-accounts.add-iam-policy-binding) PERMISSION_DENIED: Permission denied to enable service [iam.googleapis.com]
Help Token: Ae-hA1P0OMy654oi2Bv-hOZDKQ2uXlBnPAv3eohl1hgMHIXR4SsdDeBQdKeEMyzyLDgsPiWBQJiijB97QP4o75uzLNCKhGPBl6ULVo5iitljmV1A
- '@type': type.googleapis.com/google.rpc.PreconditionFailure
  violations:
  - subject: '110002'
    type: googleapis.com
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: serviceusage.googleapis.com
  reason: AUTH_PERMISSION_DENIED


In [ ]:
import os, io
import pandas as pd
from google.cloud import videointelligence
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import requests
import string
import proto
import json

## download your API Key from your Google Console

#this is the path to the API credentials. Modify this path to the json file in this folder.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Scripts/stoked-dominion-341317-24484c3fbb6f.json'
video_client = videointelligence.VideoIntelligenceServiceClient()


import difflib

In [ ]:
from google.cloud import storage


video_list = []
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):

    storage_client = storage.Client()

    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    #print("Blobs:")
    for blob in blobs:
      #print(blob.name)
      video_list.append(blob.name)




In [ ]:
list_blobs_with_prefix(
        bucket_name="videos_final", prefix="", delimiter="/"
    )

In [ ]:
def list_blobs_with_prefix1(bucket_name, prefix, out_list, delimiter=None):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    #print("Blobs:")
    for blob in blobs:
      #print(blob.name)
      out_list.append(blob.name)

### **Features Extraction**

1. speech transcription (optional) 

2. shot change detection

3. object detection

4. OCR text detection

### Get all output files under 'outputXXX/' folder in GCP





In [ ]:
labeled_vid_table={}
labeled_vid_tab=pd.read_csv('labeled_video_content.csv')
for i,v in enumerate(list(labeled_vid_tab['video_ID'])):
  labeled_vid_table[v]={}
  for q in labeled_vid_tab.columns[1:]:
    labeled_vid_table[v][q]=labeled_vid_tab.loc[i][q]

FileNotFoundError: ignored

In [ ]:
outputSpeech=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputSpeech/", delimiter="/",out_list=outputSpeech)

In [ ]:
outputDetection=[]      
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputDetection/", delimiter="/",out_list=outputDetection)

In [ ]:
outputOCR=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputOCR/", delimiter="/",out_list=outputOCR)

In [ ]:
len(outputDetection)

In [ ]:
outputLabeled=[]
list_blobs_with_prefix1(bucket_name="video_list_capstoneyt_2022", prefix="outputLabeled/", delimiter="/", out_list=outputLabeled)

###**Check duplicates** 



In [ ]:
print(len(outputLabeled))

In [ ]:
print(len(outputDetection))
print(len(outputSpeech))
print(len(outputOCR))

In [ ]:
detectionIDs=[]
for o in outputDetection:
  detectionIDs.append(o[16:-5])

In [ ]:
labeledIDs=[]
duplicates=0
for o in outputLabeled:
  labeledIDs.append(o[14:-9])

print(len(labeledIDs))


### **Feature to CSV**
1. Speech

2. Scene detection/Object detection

3. OCR texts 

**Speech**

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')



#transition words and summary words
transition='Accordingly, as a result, and so, because, consequently, for that reason, hence, on account of, since, \
    therefore, thus, after, afterwards, always, at length, during, earlier, following, immediately, in the meantime, later, \
    never, next, once, simultaneously, so far, sometimes, soon, subsequently, then, this time, until now, when, whenever, while, \
    additionally, again, also, and, or, not, besides, even more, finally, first, firstly, further, furthermore, in addition, \
    in the first place, in the second place, last, lastly, moreover, next, second, secondly, after all, although, and yet, \
    at the same time, but, despite, however, in contrast, nevertheless, notwithstanding, on the contrary, on the other hand, \
    otherwise, thought, yet, as an illustration, e.g., for example, for instance, specifically, to demonstrate, to illustrate, \
    briefly, critically, foundationally, more importantly, of less importance, primarily, above, centrally, opposite to, \
    adjacent to, below, peripherally, below, nearby, beyond, in similar fashion, in the same way, likewise, in like manner, \
    i.e., in other word, that is, to clarify, to explain, in fact, of course, undoubtedly, without doubt, surely, indeed, \
    for this purpose, so that, to this end, in order that, to that end'.split(', ')


summary=['finally', 'in a word', 'in brief', 'briefly', 
             'in conclusion', 'in the end', 'in the final analysis', 'on the whole', 'thus', 'to conclude', 'to summarize', 
             'in sum', 'to sum up', 'in summary','lastly']
             
transition=[word.strip(' ') for word in transition]



pos_tags='VB, VBD, VBG, VBP, VBZ'.split(', ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


helper functions

In [ ]:
#return tags from words (ordered)
def tags_from_corpus(words):
  tag = nltk.pos_tag(words)
  #print(tag)
  tags=[t[1] for t in tag]
  return tags
   

#count the number of occurrences of the words in word_list in corpus_list
def occurrences(corpus_list,word_list):
  res=0
  for t in corpus_list:
    if t in word_list:
      res+=1
  return res 

Speech for the already labeled vidoes (json file uploaded by Xiao)

In [ ]:
labeled_vid_table={}
corpuses={}
storage_client = storage.Client()
bucket = storage_client.get_bucket('video_list_capstoneyt_2022')

In [ ]:
%ls

In [ ]:
!pip3 install readability

In [ ]:
import readability

for vid in outputLabeled:
  # get bucket data as blob
  blob = bucket.blob(vid)
  vid_file=vid[14:-5]
  vid_id=vid[14:-9]
  try:
    labeled_vid_table[vid_id]['transcript_sentence_count']+=0
  except:
    print('===========================================================\nVideo: %s'%vid)
    #downloaded_blob = blob.download_as_string()
    #downloaded_blob = downloaded_blob.decode("utf-8") 
    # parse x:
    #y = json.loads(downloaded_blob)
    y1=open('Labeled GCP Output (Xiao)/'+vid_file+'.json')
    y=json.load(y1)
    labeled_vid_table[vid_id]={}
    #audio transcription 
    try:
      try:
        atn=y[vid_file][0]['speech']['annotationResults'][0]['speechTranscriptions']
      except:
        try:
          atn=y['annotation_results'][0]['speech_transcriptions']
        except:
          atn=y['annotation_results'][1]['speech_transcriptions']

      

      corpus=""
      #frequencies of word
      screen_time={}
      #confidence score of that entity 
      conf=[]

      for i in range(len(atn)):
        if 'transcript' in atn[i]['alternatives'][0].keys() and 'confidence' in atn[i]['alternatives'][0].keys():
          corpus+=(atn[i]['alternatives'][0]['transcript']+"\n")
          conf.append(atn[i]['alternatives'][0]['confidence'])
      labeled_vid_table[vid_id]['transcript_sentence_count']=len(corpus.split('.'))
      labeled_vid_table[vid_id]['transcription_confidence_avg']=np.mean(conf)
      
      try:
        results = readability.getmeasures(corpus, lang='en')
        labeled_vid_table[vid_id]['readability']= results['readability grades']['ARI']
      except:
        print('readability unavailable!')

      #word counts
      words=nltk.word_tokenize(corpus)
      corpuses[vid_id]=corpus

      labeled_vid_table[vid_id]['transcript_wordcount']=len(words)
      labeled_vid_table[vid_id]['transcript_unique_wordcount']=len(set(words))  

      labeled_vid_table[vid_id]['transcription_active_wordcount']=occurrences(tags_from_corpus(words),pos_tags)
      labeled_vid_table[vid_id]['transcription_summary_wordcount']=occurrences(words,summary)
      labeled_vid_table[vid_id]['transcription_transition_wordcount']=occurrences(words,transition)
      #https://py-readability-metrics.readthedocs.io/en/latest/

      print('transcript features added.')

    except:
      print('speech annotation does not exist!')

In [ ]:
for c in list(corpuses.keys()):
  with open('text/transcript_text/labeled/raw_text/%s.txt'%c,'w') as f:
    f.write(corpuses[c])

For the speech processing of our **new videos**

In [ ]:
corpuses_unlabeled={}
vid_table={}

In [ ]:
import readability

for vid in outputSpeech:
  # get bucket data as blob
  blob = bucket.blob(vid)
  vid_id=vid[13:-5]
  try:
    vid_table[vid_id]['transcript_sentence_count']+=0
  except:
    print('===========================================================\nVideo: %s'%vid)
    downloaded_blob = blob.download_as_string()
    downloaded_blob = downloaded_blob.decode("utf-8") 
    # parse x:
    y = json.loads(downloaded_blob)
    if vid_id not in vid_table.keys():
      vid_table[vid_id]={}
    #audio transcription 
    if True:
    #try:
      atn=y['annotation_results'][0]['speech_transcriptions']

      corpus=""
      #frequencies of word
      screen_time={}
      #confidence score of that entity 
      conf=[]

      for i in range(len(atn)):
        if 'transcript' in atn[i]['alternatives'][0].keys(): 
          corpus+=(atn[i]['alternatives'][0]['transcript']+"\n")
        if 'confidence' in atn[i]['alternatives'][0].keys():
          conf.append(atn[i]['alternatives'][0]['confidence'])
      vid_table[vid_id]['transcript_sentence_count']=len(corpus.split('.'))
      vid_table[vid_id]['transcription_confidence_avg']=np.mean(conf)
      try:
        results = readability.getmeasures(corpus, lang='en')
        vid_table[vid_id]['readability']= results['readability grades']['ARI']
      except:
        print('readability unavailable!')

      #word counts
      words=nltk.word_tokenize(corpus)
      corpuses_unlabeled[vid_id]=corpus

      vid_table[vid_id]['transcript_wordcount']=len(words)
      vid_table[vid_id]['transcript_unique_wordcount']=len(set(words))  

      vid_table[vid_id]['transcription_active_wordcount']=occurrences(tags_from_corpus(words),pos_tags)
      vid_table[vid_id]['transcription_summary_wordcount']=occurrences(words,summary)
      vid_table[vid_id]['transcription_transition_wordcount']=occurrences(words,transition)
      #https://py-readability-metrics.readthedocs.io/en/latest/

      print('transcript features added.')

    else:
      #print(e)
      print('speech annotation does not exist!')
      print(str(y['annotation_results'][0]['speech_transcriptions'][2]['alternatives'][0]))
      break

Transcript data from additional resources - speech2txt.py 

In [ ]:
file_names = os.listdir('text/transcript_text/unlabeled/raw_text/')
for f in file_names:
  if f[-3:]=='txt':
    vid_id=f[:-4]
    if vid_id not in vid_table.keys():
      vid_table[vid_id]={}

    try:
      vid_table[vid_id]['transcript_wordcount'].split('')
    except:
      x=open('text/transcript_text/unlabeled/raw_text/%s'%f).readlines()
      if len(x)>0:
        corpus=x[0].lower()
        
        vid_table[vid_id]['transcript_sentence_count']=len(corpus.split('.'))
        corpuses_unlabeled[vid_id]=corpus
        try:
          results = readability.getmeasures(corpus, lang='en')
          vid_table[vid_id]['readability']= results['readability grades']['ARI']
        except:
          print('readability unavailable!')
        #word counts
        words=nltk.word_tokenize(corpus)

        vid_table[vid_id]['transcript_wordcount']=len(words)
        vid_table[vid_id]['transcript_unique_wordcount']=len(set(words))  

        vid_table[vid_id]['transcription_active_wordcount']=occurrences(tags_from_corpus(words),pos_tags)
        vid_table[vid_id]['transcription_summary_wordcount']=occurrences(words,summary)
        vid_table[vid_id]['transcription_transition_wordcount']=occurrences(words,transition)

  if f[-4:]=='json':
    vid_id=f[:-5]
    try:
      vid_table[vid_id]['test']=0
    except:
      vid_table[vid_id]={}
    y=open('text/transcript_text/unlabeled/raw_text/%s'%f).read()
    x=json.loads(y)
    conf=[]

    for i in range(len(x)):
      conf.append(x[i]['confidence'])
      
    vid_table[vid_id]['transcription_confidence_avg']=np.mean(conf)

In [ ]:
for c in list(corpuses_unlabeled.keys()):
  with open('text/transcript_text/unlabeled/raw_text/%s.txt'%c,'w') as f:
    f.write(corpuses_unlabeled[c])

**Detection**

for Xiao's videos

In [ ]:
for vid in outputLabeled:
  # get bucket data as blob
  
  vid_file=vid[14:-5]
  vid_id=vid[14:-9]
  try:
    labeled_vid_table[vid_id]['unique_obj_count']+=0
    labeled_vid_table[vid_id]['scene_count']+=0
  except:
    #object and shot detection
    try:
      blob = bucket.blob(vid)
      print('===========================================================\nVideo: %s'%vid)
      #downloaded_blob = blob.download_as_string()
      #downloaded_blob = downloaded_blob.decode("utf-8") 
      # parse x:
      #y = json.loads(downloaded_blob)
      y1=open('Labeled GCP Output (Xiao)/'+vid_file+'.json')
      y=json.load(y1)
      #object 
      try:
        atn1=y[vid_file][0]['object']['annotationResults'][0]['objectAnnotations']
      except:
        try:
          atn1=y[vid_file][0]['objects']['annotationResults'][0]['objectAnnotations']
        except:
          try:
            atn1=y['annotation_results'][0]['object_annotations']
          except:
            atn1=y['annotation_results'][1]['object_annotations']
      #scene- the lengths of the array is the # of scenes
      try:
        atn2=y[vid_file][0]['shots']
      except:
        try:
          atn2=y['annotation_results'][0]['shot_annotations']
        except:
          atn2=y['annotation_results'][1]['shot_annotations']
      
      #frequencies of word
      obj_screen_time={}
      #confidence score of that entity 
      obj_conf=[]

      for i in range(len(atn1)):
        if 'confidence' in atn1[i].keys() and 'entity' in atn1[i].keys() and 'description' in atn1[i]['entity'].keys():
          if atn1[i]['entity']['description'] not in obj_screen_time.keys():
            obj_screen_time[atn1[i]['entity']['description']]=1
          else:
            obj_screen_time[atn1[i]['entity']['description']]+=1
          obj_conf.append([atn1[i]['entity']['description'],atn1[i]['confidence']])
      
      conf_scores=np.array([c[1] for c in obj_conf])

      labeled_vid_table[vid_id]['unique_obj_count']=len(obj_screen_time)
      labeled_vid_table[vid_id]['scene_count']=len(atn2)

      #plt.hist(conf_scores,bins='auto')
      print('object and shot detection features added.')


    except:
      print('object or shot detection does not exist!')
      #print(str(y['annotation_results'][0].keys()))
      #try:
      #  print(str(y[vid_file]['annotation_results'])[:500])
      #except:
      #  break


      #break


for our videos 

In [ ]:
for vid in outputDetection:
  # get bucket data as blob
  
  vid_id=vid[16:-5]
  try:
    vid_table[vid_id]['unique_obj_count']+=0
    vid_table[vid_id]['scene_count']+=0
  except:
    #object and shot detection
    if vid_id not in vid_table.keys():
      vid_table[vid_id]={}
    try:
      blob = bucket.blob(vid)
      print('===========================================================\nVideo: %s'%vid)
      downloaded_blob = blob.download_as_string()
      downloaded_blob = downloaded_blob.decode("utf-8") 
      # parse x:
      y = json.loads(downloaded_blob)
      #object 
      atn1=y['annotation_results'][0]['object_annotations']

      #scene- the lengths of the array is the # of scenes
      atn2=y['annotation_results'][0]['shot_annotations']
      
      #frequencies of word
      obj_screen_time={}
      #confidence score of that entity 
      obj_conf=[]

      for i in range(len(atn1)):
        if atn1[i]['entity']['description'] not in obj_screen_time.keys():
          obj_screen_time[atn1[i]['entity']['description']]=1
        else:
          obj_screen_time[atn1[i]['entity']['description']]+=1
        obj_conf.append([atn1[i]['entity']['description'],atn1[i]['confidence']])
      
      conf_scores=np.array([c[1] for c in obj_conf])
      if vid_id in vid_table.keys():
        vid_table[vid_id]['unique_obj_count']=len(obj_screen_time)
        vid_table[vid_id]['scene_count']=len(atn2)
      else:
        vid_table[vid_id]={}
        vid_table[vid_id]['unique_obj_count']=len(obj_screen_time)
        vid_table[vid_id]['scene_count']=len(atn2)   

      #plt.hist(conf_scores,bins='auto')
      print('object and shot detection features added.')


    except:
      print('object or shot detection does not exist!')
      print(str(y['annotation_results'][0].keys())[:400])


**OCR**

OCR for Xiao's video 

In [ ]:
for vid in outputLabeled:
  # get bucket data as blob
  
  #concatenate video ID
  vid_file=vid[14:-5]
  vid_id=vid[14:-9]
  try:
    labeled_vid_table[vid_id]['OCR_confidence_avg']+=0
  except:
    blob = bucket.blob(vid)
    print('===========================================================\nVideo: %s'%vid)
    #downloaded_blob = blob.download_as_string()
    #downloaded_blob = downloaded_blob.decode("utf-8") 
    # parse x:
    #y = json.loads(downloaded_blob)
    y1=open('Labeled GCP Output (Xiao)/'+vid_file+'.json')
    y=json.load(y1)
    #OCR (visul texts) annotations
    try:
      try:
        atn3=y[vid_file][0]['text']['annotationResults'][0]['textAnnotations']
      except:
        try:
          atn3=y['annotation_results'][0]['text_annotations']
        except:
          try:
            atn3=y['annotation_results'][1]['text_annotations']
          except:
            atn3=y[vid_file][0]['text']
      #frequencies of word
      screen_time={}
      #confidence score of that entity 
      conf=[]
      for i in range(len(atn3)):
        conf.append([atn3[i]['text'],atn3[i]['segments'][0]['confidence']])

      #confidence scores
      conf_scores=np.array([c[1] for c in conf]) 

      #add confidence score as a feature to vid_table
      labeled_vid_table[vid_id]['OCR_confidence_avg']=np.mean(conf_scores)

      #plt.hist(conf_scores,bins='auto')
      
      print('OCR text annotation features added.')

    except:
      print('text annotation does not exist!')
      print(str(y.keys())[:400])
      #break


OCR for our videos

In [ ]:
for vid in outputOCR:
  # get bucket data as blob
  
  #concatenate video ID
  vid_id=vid[10:-5]

  try:
    vid_table[vid_id]['OCR_confidence_avg']+=0
    print(vid_table[vid_id]['OCR_confidence_avg'])
    print(vid_id)
  except:
    if vid_id not in vid_table.keys():
      vid_table[vid_id]={}
    blob = bucket.blob(vid)
    print('===========================================================\nVideo: %s'%vid)
    downloaded_blob = blob.download_as_string()
    downloaded_blob = downloaded_blob.decode("utf-8") 
    # parse x:
    y = json.loads(downloaded_blob)
    #OCR (visul texts) annotations
    try:
      atn3=y['annotation_results'][0]['text_annotations']
      #frequencies of word
      screen_time={}
      #confidence score of that entity 
      conf=[]
      for i in range(len(atn3)):
        if 'text' in atn3[i].keys() and 'segments' in atn3[i].keys():
          conf.append([atn3[i]['text'],atn3[i]['segments'][0]['confidence']])

      #confidence scores
      conf_scores=np.array([c[1] for c in conf]) 

      #add confidence score as a feature to vid_table
      if vid_id in vid_table.keys():
        print('processed')
        vid_table[vid_id]['OCR_confidence_avg']=np.mean(conf_scores)
      else:
        vid_table[vid_id]={}
        vid_table[vid_id]['OCR_confidence_avg']=np.mean(conf_scores)
      
      print('OCR text annotation features added.')

    except:
      print('text annotation does not exist!')
      print(y['annotation_results'][0]['segment'])
      


**transform map to pandas dataframe**

**Xiao's json files** 

In [ ]:
cols=labeled_vid_table['-71ld0iqAq8'].keys()
idx=labeled_vid_table.keys()
labeled_vid_table_pd=pd.DataFrame(columns=cols,index=idx)

for i,id in enumerate(idx):
  for col in cols:
    if col in labeled_vid_table[id].keys():
      labeled_vid_table_pd.iloc[i][col]=labeled_vid_table[id][col]

In [ ]:
labeled_vid_table_pd=labeled_vid_table_pd.reset_index()
#
#json file 
labeled_vid_table_pd=labeled_vid_table_pd.rename(columns={'index':'video_ID'})
labeled_vid_table_pd=labeled_vid_table_pd.set_index('video_ID')
#labeled_vid_table_pd=labeled_vid_table_pd.drop(columns='test')
labeled_vid_table_pd

In [ ]:
labeled_vid_table_pd.to_csv('Content Features Table (Labeled data - Xiao)/labeled_video_raw_content.csv')

**features of our video**

In [ ]:
print(vid_table['z4Om-v-ZhLI']['OCR_confidence_avg'])

In [ ]:
cols=vid_table['-iL5AeXjpgg'].keys()
idx=vid_table.keys()
vid_table_pd=pd.DataFrame(columns=cols,index=idx)
print(len(vid_table))

for i,id in enumerate(idx):
  #filter out the duplicates in LabeledIDs
  if id in detectionIDs and i not in labeledIDs:
    for col in cols:
      if col in vid_table[id].keys():
        vid_table_pd.iloc[i][col]=vid_table[id][col]

In [ ]:
print(cols)

In [ ]:
vid_table_pd=vid_table_pd.reset_index()
#json file 
vid_table_pd=vid_table_pd.rename(columns={'index':'video_ID'})
vid_table_pd=vid_table_pd.set_index('video_ID')
#vid_table_pd=vid_table_pd.drop(columns='test')
#vid_table_pd.dropna()

In [ ]:
vid_table_pd

In [ ]:
vid_table_pd.to_csv('Content Features Table (Unlabeled data) /video_raw_content.csv')